# Wealth Model 3
## Here we provide the first tax model

We shall introduce a population of N agents, each with equal share of wealth of the nation (\$1,000).

They encounter each other randomly and exchange a random amount of their wealth in each exchange, with the direction being random. 

Then we tax the "income" of agents at regular intervals (to be fixed) and redistribute this flat tax among all agents equally. The equilibrium will be probably again an exponential distribution, but reached much slower.

## Preparation

In [ ]:
using Distributions
using Plots

### Function Definitions

In [ ]:
function Gini(A::Vector)::Float64
    gi=0.
    for i=1:length(A)
        for j=i+1:length(A)
            gi+=abs(A[i]-A[j])
        end
    end
    g=gi/(length(A)*sum(A))
    return g
end

In [ ]:
function WealthTopXPerc(A::Vector,x::Float64)::Float64 
    s=zeros(Int64,length(A))
    s=sort(A)
    indmin=Int(round(x*length(A)))
    w=0
    for i=length(A)-indmin:length(A)
        w+=s[i]
    end
    wf=w*100.0/sum(A)   # in percent of total
    return wf
end

In [ ]:
function VectorToDistr(A::Vector,k::Int)::Vector
    d=zeros(Int64,k)
    b=maximum(A)
    for i=1:length(A)
        for j=1:k
            if (A[i] >= (j-1)*b/k)
               if (A[i] <= j*b/k) 
                  d[j]=d[j]+1
               end
            end
        end
    end
    return d
end

In [ ]:
function AgentsInteractAmount(A::Vector)
    x=rand(big.(1:length(A)),2)
    if (A[x[1]] > A[x[2]])
        diff=A[x[2]]
    else
        diff=A[x[1]]
    end
#    print("\n",diff)
    y=rand(2)
    if (y[1] < .5) 
        A[x[1]]+=Int(round(y[2]*diff))
        A[x[2]]-=Int(round(y[2]*diff))
    else
        A[x[1]]-=Int(round(y[2]*diff))
        A[x[2]]+=Int(round(y[2]*diff))
   end
end

This next function was erroneously thought to be an income tax. It is a wealth tax and will be studied in another notebook. So I'll remove it here.

In [ ]:
function TaxRate(A,B::Vector)::Vector
    R=zeros(Float64,length(A))
    for i=1:length(A)
        R[i]=(B[i]-A[i])/B[i]
    end
    return R
end

Here is an income tax which is distributed to the best of possibility back to all agents. It is currently a flat tax of a certain percentage. Losses are not allowed to be taken into account.

In [ ]:
function TaxAgentsIncome(A::Vector,B::Vector,tax::Float64)
    sv=sum(A)
    for i=1:length(A)
        earn=A[i]-B[i]
        t=0
        if (earn > 0)
            t=Int(round(earn*tax))   # Only positive income is taxed
        else
            t=0
        end
        A[i]-=t   
    end
    sn=sum(A)
    total=sv-sn
#    print("\n sv ",sv," sn ",sn," total ",total," to distr to each", floor(total/length(A)))
#    print("\n real vs int values ", total/length(A)," ",Int(round(total/length(A))))
    zuf=rand(length(A))
    for i=1:length(A)
        A[i]+=Int(floor(total/length(A)))
        if (zuf[i] < (total/length(A)-floor(total/length(A))))
            A[i]+=1
        end
    end
end

## First - the regular flat income tax regime: 30% 

### Initialization

In [ ]:
using Random 
Random.seed!(1);
N=1000  # Number of Agents
W=1000   # Starting share of wealth for all
T=100000  # Number of iterations
K=10  # Number of bins for the wealth distribution
tax=0.3   # Percentage of flat income tax (30 (it), 60 (sit), 5 (wit))
taxperiod=10   # Every taxperiod iterations, taxes are due and redistributed
R=[0,1,10,100,1000,10000,100000]    # Reporting times T=...
b=W/K
print("Number of bins for distribution: ",K," bin size ",b)
A = fill(W,N);
B = fill(W,N);
O = fill(0,T);
M = fill(0.0,T);
I = fill(0,T);
Rate = fill(0,N);
wealth = fill(0.0,T,3)
D = fill(0,K);
g = fill(0.0,T,3);
D=VectorToDistr(A,K);

### Start Iterations with Regular Income Tax of 30%

In [ ]:
ic=1
D_label="t=" * string(R[ic])
display(bar(D, func=cdf, alpha=0.3, legend=:topleft, label = D_label, xlabel="Income Bin", ylabel="Number of Agents", xticks=1:10))
#
ic=2    # This is the first report after the initial distribution
Om=0
for i=1:T
    AgentsInteractAmount(A)
    I[i]=i
    if (Om < maximum(A)) ## Modified to maximum across all iterations so far
        O[i]=maximum(A)
        Om=maximum(A)
    else
        O[i]=O[i-1]
    end
    M[i]=median(A)
    wealth[i,1]=100.0-WealthTopXPerc(A,.5)
    wealth[i,2]=WealthTopXPerc(A,.1)
    wealth[i,3]=WealthTopXPerc(A,.01)    
    if (mod(i,taxperiod) == 0)
        TaxAgentsIncome(A,B,tax)
        Rate=TaxRate(A,B)*100.
        for j=1:length(A)
            B[j]=A[j]
        end
    end
    g[i,1]=Gini(A)
    if (i == R[ic])
       D=VectorToDistr(A,K)
#       print("\n D: ",D," Iteration ",i)
       D_label="t=" * string(R[ic]) * "   Flat Income Tax"
       if (i > 500)
          display(bar(D, func=cdf, alpha=0.3, legend=:topright, label = D_label, xlabel="Income Bin", ylabel="Number of Agents", xticks=1:K))
       else
           display(bar(D, func=cdf, alpha=0.3, legend=:topleft, label = D_label, xlabel="Income Bin", ylabel="Number of Agents", xticks=1:K))
       end           
       display(scatter(sort(A),sort(Rate), legend=:bottomright, xlabel="Income in \$", ylabel="Tax in %", label = "Effective Tax Rate", markersize = 3, c = :orange))
       ic+=1
    end
end
print("\n Minimum ", minimum(A), "\n Maximum ", maximum(A), "\n Sum ", 
    sum(A))
plot(I,O,legend=:topleft,label = ["Max Wealth"], xlabel="Iterations t", ylabel="Wealth in \$")
display(plot!(I,M,legend=:topleft,label = ["Median Wealth"]))
display(plot(I,wealth,legend=:topright,xlabel="Iterations t",ylabel="Share of Wealth in %",label = ["Wealth Bottom 50%" "Wealth Top 10%" "Wealth Top 1%"]))

In [ ]:
display(plot(I,wealth,legend=:best,xlabel="Iterations t",ylabel="Share of Wealth in %",label = ["Wealth Bottom 50%" "Wealth Top 10%" "Wealth Top 1%"]))